In [2]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table
from dash.dependencies import Input, Output


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pymongo import MongoClient

# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from CR_Functions import AnimalShelter



###########################
# Data Manipulation / Model
###########################
# Change to your own username, password, port, and CRUD Python module name

username = "aacuser"
password = "userjojo130151"
port = 38610
shelter = AnimalShelter(username, password, port)


# class read method must support return of cursor object and accept projection json input
df = pd.DataFrame.from_records(shelter.read({}))



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

app.layout = html.Div([
    # Header with student name and class name
    html.Center(html.B(html.H2('Jordan Davis - SNHU CS-340 Dashboard'))),
    html.Hr(),
    # Dropdown for filtering rescue dog preferences
    dcc.Dropdown(
        id='dropdown-id',
        options=[{'label': 'Water Rescue', 'value': 'water'},
                 {'label': 'Mountain Rescue', 'value': 'mountain'},
                 {'label': 'Disaster Rescue', 'value': 'disaster'},
                 {'label': 'Reset', 'value': 'reset'}],
        value='Reset',
        placeholder='What kind of rescue animal are you looking for?'
    ),
    # Datatable to display all animal's information in table format
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in sorted(df.columns)
        ],
        data=df.to_dict('records'),
    
        # Styling and functionality for DataTable
        row_selectable=False,
        column_selectable=False,
        row_deletable=False,
        editable=False,
        selected_columns=[],
        selected_rows=[],
        page_action='native',
        page_current=0,
        page_size=6,
        
        style_cell={
            'textalign': 'left',
            'overflow': 'hidden',
            'textOverflow': 'ellipsis',
            'maxWidth': 20,
        },
        
        style_data_conditional=[
            {
            'if': {'row_index': 'even'},
            'backgroundColor': 'rgb(220, 220, 220)',
            },
            
        ],
        
        style_data={
            'color': 'blue',
            'backgroundColor': 'white',
            'border': '1px solid black',
        },
        
        style_header={
            'backgroundColor': 'rgb(200, 200, 200)',
            'color': 'black',
            'fontWeight': 'semi bold',
            'border': '1px solid black'
        },
        
        # Tooltip to help display full cell text
        tooltip_data=[
            {
                column: {'value': str(value), 'type': 'markdown'}
                for column, value in row.items()
            } for row in df.to_dict('records')
        ],
        
        tooltip_delay=0,
        tooltip_duration=None,
    ),
    
    html.Br(),
     html.Hr(),
    html.Div(style={'display':'flex'}, children=[
        # Geolocation Map
        html.Div(
            id='map-id',
            className='col s12 m6'
        ),
        # Pie graph for breeds
        html.Div(
            id='pie-id'
        ),
        html.Div(
        html.Img(src="https://learn.snhu.edu/content/enforced/1069845-CS-340-T5629-OL-TRAD-UG.22EW5/course_documents/Grazioso%20Salvare%20Logo.png",
             title='https://learn.snhu.edu/content/enforced/1069845-CS-340-T5629-OL-TRAD-UG.22EW5/course_documents/Grazioso%20Salvare%20Logo.png',
             height=200,
             width=220),
        )
    ]),
    
    
])



#############################################
# Interaction Between Components / Controller
#############################################


#This callback will highlight a row on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')])
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

# Geolocation map - Display a popup and tooltip with Animal type and name
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    return [
        dl.Map(style={'width': '450px', 'height': '400px'}, center=[30.75, -97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            #Marker with tool tip and popup
            dl.Marker(position=[30.75, -97.48], children=[
                dl.Tooltip(dff.iloc[1,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[1,10])
                ])
            ])
        ])
    ]

# Updata data based on dropdown selection
@app.callback(
    [Output('datatable-id', 'data'),
     Output('datatable-id', 'columns')],
    [Input('dropdown-id', 'value')])
def update_query(value):
    if value == 'water': # Water rescue preferred breeds, sex, and age
        new = pd.DataFrame.from_records(shelter.read(
            {'$and': [
                {'animal_type': 'Dog'},
                {'$or': [
                    {'breed': 'Labrador Retriever Mix'},
                    {'breed': 'Chesapeake Bay Retriever'},
                    {'breed': 'Newfoundland'}
                ]},
                {'sex_upon_outcome': 'Intact Female'},
                {'age_upon_outcome_in_weeks': {'$gte': 26, '$lte': 156}}
            ]}
        ))
    elif value == 'mountain': # Mountain rescue preferred breeds, sex, and age
        new = pd.DataFrame.from_records(shelter.read(
            {'$and': [
                {'animal_type': 'Dog'},
                {'$or': [
                    {'breed': 'German Shepard'},
                    {'breed': 'Alaskan Malamute'},
                    {'breed': 'Old English Sheepdog'},
                    {'breed': 'Siberian Husky'},
                    {'breed': 'Rottweiler'}
                ]},
                {'sex_upon_outcome': 'Intact Male'},
                {'age_upon_outcome_in_weeks':  {'$gte': 26, '$lte': 156}}
            ]}
        ))
    elif value == 'disaster': # Disaster rescue preferred breeds, sex, and age
        new = pd.DataFrame.from_records(shelter.read(
            {'$and': [
                {'animal_type': 'Dog'},
                {'$or': [
                    {'breed': 'Doberman Pinscher'},
                    {'breed': 'German Shepard'},
                    {'breed': 'Golden Retriever'},
                    {'breed': 'Bloodhound'},
                    {'breed': 'Rottweiler'}
                ]},
                {'sex_upon_outcome': 'Intact Male'},
                {'age_upon_outcome_in_weeks':  {'$gte': 20, '$lte': 300}}
            ]}
        ))
    elif value == 'reset': # Reset to unfilter data
        new = pd.DataFrame.from_records(shelter.read({}))
    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in sorted(new.columns)]
    data=new.to_dict('records')
    return(data, columns) # Return data and column variables to datatable

# Pie Graph - display percentage of breeds
@app.callback(
    Output('pie-id', "children"),
    [Input('datatable-id', "data")])
def update_graph(data):
    fig = px.pie(data, names='breed', title='Breeds')
    return dcc.Graph(figure=fig)
    
app